# Finding the heavy traffic indicators on I-94

![I-94 Figma Overview](i94_figma.png)



## Introduction
Interstate 94 is an east-west Interstate Highway in United States. In this project, we are going to analyze the data to **determine the key indicators of heavy traffic** on I-94. These indicators can be weather type, time of the day, etc.

## Dataset
The dataset we are using is from UCI Machine Learning Repository, it includes all the necessary details of traffic on I-94. The data is recorded by the station located between Minneapolis and Saint Paul, and only the traffic moving **from east to west** is recorded, but is sufficient enough for this analysis.

## Examine the dataset

In [1]:
# import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
# load the dataset
df = pd.read_csv("Metro_Interstate_Traffic_Volume.csv")

In [11]:
# display the general information about the DataFrame
print("*" * 100)
print("General Information:")
print(df.info())
print("*" * 100)

# print the first and last 5 rows of the DataFrame
print("*" * 100)
print("First 5 rows:")
print(df.head())
print("*" * 100)

print("*" * 100)
print("Last 5 rows:")
print(df.tail())
print("*" * 100)

****************************************************************************************************
General Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48204 entries, 0 to 48203
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   holiday              61 non-null     object 
 1   temp                 48204 non-null  float64
 2   rain_1h              48204 non-null  float64
 3   snow_1h              48204 non-null  float64
 4   clouds_all           48204 non-null  int64  
 5   weather_main         48204 non-null  object 
 6   weather_description  48204 non-null  object 
 7   date_time            48204 non-null  object 
 8   traffic_volume       48204 non-null  int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 3.3+ MB
None
****************************************************************************************************
****************************************************

There are several interesting observations about the dataset. First, it includes **9 columns** with different features, such as **numerical information like temperature, rain, and snow**, as well as **descriptive information like weather description**. The "holiday" column indicates **whether a day is a holiday**; only 61 entries are non-null, which means there are only 61 holidays in the dataset.